In [1]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
pip install -e .


Obtaining file:///C:/Users/Mukil/ai_safety_lib
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for ai-safety-lib (pyproject.toml): started
  Building editable for ai-safety-lib (pyproject.toml): finished with status 'done'
  Created wheel for ai-safety-lib: filename=ai_safety_lib-0.2.0-0.editable-py3-none-any.whl size=7883 sha256=0c57c37f587742289f78f719fa40243b3fb7a3ad964262951aea95c13afdd5ce
  Stored in directory: C:\Users\Mukil\AppData\Local\Temp\pip-ephem-wheel-cache-sghha50z\wheels\6e\39\4a\9c775ca733893d701b39a9f1e53df68

In [3]:
from ai_safety_lib import SafetyGate
print("AI Safety Library loaded")


AI Safety Library loaded


In [4]:
!pip install scikit-learn


In [5]:
import numpy as np
import sklearn
print("NumPy:", np.__version__)
print("Scikit-learn:", sklearn.__version__)



NumPy: 2.4.2
Scikit-learn: 1.8.0


In [6]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [7]:
data = load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


In [8]:
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(max_iter=500)
model.fit(X_train_scaled, y_train)

print("Model trained successfully")


Model trained successfully


In [9]:
sample = X_test_scaled[0].reshape(1, -1)
probs = model.predict_proba(sample)[0]

confidence = float(np.max(probs))
entropy = float(-np.sum(probs * np.log(probs + 1e-8)))

model_output = {
    "confidence": confidence,
    "entropy": entropy
}

print("=== NORMAL INPUT ===")
print("Confidence:", confidence)
print("Entropy:", entropy)


=== NORMAL INPUT ===
Confidence: 0.8820408955973951
Entropy: 0.3628388100635645


In [10]:
drifted_sample = sample + np.random.normal(0, 1.5, sample.shape)

probs_drifted = model.predict_proba(drifted_sample)[0]

confidence_drifted = float(np.max(probs_drifted))
entropy_drifted = float(-np.sum(probs_drifted * np.log(probs_drifted + 1e-8)))

drifted_output = {
    "confidence": confidence_drifted,
    "entropy": entropy_drifted
}

print("=== DRIFTED INPUT ===")
print("Confidence:", confidence_drifted)
print("Entropy:", entropy_drifted)


=== DRIFTED INPUT ===
Confidence: 0.9983572360106597
Entropy: 0.012173770043666397


In [ ]:
from ai_safety_lib import SafetyGate
gate = SafetyGate(confidence_threshold=0.6)
print("SafetyGate initialized successfully")


SafetyGate initialized successfully


In [15]:
gate = SafetyGate(confidence_threshold=0.6)

# Format the data for SafetyGate
predictions = probs_drifted.tolist()

# Convert numpy arrays to dictionary format with feature names
feature_names = [f"feature_{i}" for i in range(sample.shape[1])]
reference_data = {name: [sample[0, i]] for i, name in enumerate(feature_names)}
current_data = {name: [drifted_sample[0, i]] for i, name in enumerate(feature_names)}

assessment = gate.evaluate(
    predictions=predictions,
    reference_data=reference_data,
    current_data=current_data
)

print("=== AI SAFETY LIBRARY DECISION ===")
print(f"Overall Risk: {assessment.overall_risk}")
print(f"Risk Level: {assessment.risk_level}")
print(f"Should Deploy: {gate.should_deploy(assessment)}")
print(f"\nComponent Risks: {assessment.component_risks}")


=== AI SAFETY LIBRARY DECISION ===
Overall Risk: 0.5
Risk Level: SafetyLevel.WARNING
Should Deploy: False

Component Risks: {'confidence': 0.5, 'drift': 1.0}


In [ ]:
print("=== TEMPORAL RISK TREND ===")

risk_scores = []
for i in range(5):
    assessment = gate.evaluate(
        predictions=predictions,
        reference_data=reference_data,
        current_data=current_data
    )
    risk_scores.append(assessment.overall_risk)
    print(f"Iteration {i+1} → Risk Score: {assessment.overall_risk:.4f}, Level: {assessment.risk_level}")


=== TEMPORAL RISK TREND ===


TypeError: SafetyGate.evaluate() got an unexpected keyword argument 'model_output'

In [ ]:
!pip install matplotlib seaborn

## 📊 Comprehensive Comparison: With vs Without AI Safety Library

This section demonstrates the value of the AI Safety Library by:

1. **Generating test samples** with varying levels of data drift
2. **Comparing predictions** with and without safety checks
3. **Visualizing the differences** in 4 comprehensive charts:
   - Model confidence degradation 
   - Risk score assessment
   - Deployment decision comparison
   - Safety-based filtering visualization

Run the cells below to see how the library protects against deploying unsafe predictions!

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)

In [ ]:
# Generate multiple test samples with varying drift levels
print("Generating test samples with varying drift levels...\n")

num_samples = 50
drift_levels = np.linspace(0, 3, num_samples)  # From no drift to high drift

results_without_library = []
results_with_library = []

for drift_level in drift_levels:
    # Create drifted sample
    test_sample = sample + np.random.normal(0, drift_level, sample.shape)
    
    # Get model predictions WITHOUT safety library
    probs_test = model.predict_proba(test_sample)[0]
    pred_class = np.argmax(probs_test)
    confidence_test = float(np.max(probs_test))
    
    results_without_library.append({
        'drift_level': drift_level,
        'confidence': confidence_test,
        'prediction': pred_class,
        'deployed': True  # Without library, everything gets deployed
    })
    
    # Evaluate WITH safety library
    predictions_test = probs_test.tolist()
    current_data_test = {name: [test_sample[0, i]] for i, name in enumerate(feature_names)}
    
    assessment_test = gate.evaluate(
        predictions=predictions_test,
        reference_data=reference_data,
        current_data=current_data_test
    )
    
    should_deploy = gate.should_deploy(assessment_test)
    
    results_with_library.append({
        'drift_level': drift_level,
        'confidence': confidence_test,
        'prediction': pred_class,
        'risk_score': assessment_test.overall_risk,
        'risk_level': assessment_test.risk_level.value,
        'deployed': should_deploy
    })

print(f"Generated {num_samples} test samples")
print(f"Drift range: {drift_levels[0]:.2f} to {drift_levels[-1]:.2f}")

In [ ]:
# Create comprehensive comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Model Behavior: With vs Without AI Safety Library', fontsize=16, fontweight='bold')

# Extract data
drift_vals = [r['drift_level'] for r in results_without_library]
conf_without = [r['confidence'] for r in results_without_library]
conf_with = [r['confidence'] for r in results_with_library]
risk_scores = [r['risk_score'] for r in results_with_library]
deployed_with = [r['deployed'] for r in results_with_library]

# Plot 1: Confidence vs Drift Level
ax1 = axes[0, 0]
ax1.plot(drift_vals, conf_without, 'o-', label='Without Safety Library', alpha=0.7, linewidth=2)
ax1.axhline(y=0.6, color='orange', linestyle='--', label='Confidence Threshold (0.6)', linewidth=2)
ax1.set_xlabel('Drift Level', fontsize=12)
ax1.set_ylabel('Model Confidence', fontsize=12)
ax1.set_title('Model Confidence Degradation with Drift', fontsize=13, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Risk Score from Safety Library
ax2 = axes[0, 1]
ax2.plot(drift_vals, risk_scores, 'o-', color='red', label='Risk Score', alpha=0.7, linewidth=2)
ax2.axhline(y=0.5, color='orange', linestyle='--', label='Warning Threshold', linewidth=2)
ax2.axhline(y=0.7, color='darkred', linestyle='--', label='Critical Threshold', linewidth=2)
ax2.set_xlabel('Drift Level', fontsize=12)
ax2.set_ylabel('Risk Score', fontsize=12)
ax2.set_title('AI Safety Library Risk Assessment', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.fill_between(drift_vals, 0, risk_scores, alpha=0.2, color='red')

# Plot 3: Deployment Decisions
ax3 = axes[1, 0]
deployed_count_without = sum([1 for r in results_without_library if r['deployed']])
deployed_count_with = sum([1 for r in results_with_library if r['deployed']])
blocked_count = num_samples - deployed_count_with

categories = ['Without\nSafety Library', 'With\nSafety Library']
deployed_counts = [deployed_count_without, deployed_count_with]
blocked_counts = [0, blocked_count]

x_pos = np.arange(len(categories))
width = 0.6

bars1 = ax3.bar(x_pos, deployed_counts, width, label='Deployed', color='green', alpha=0.7)
bars2 = ax3.bar(x_pos, blocked_counts, width, bottom=deployed_counts, label='Blocked', color='red', alpha=0.7)

ax3.set_ylabel('Number of Predictions', fontsize=12)
ax3.set_title('Deployment Decisions Comparison', fontsize=13, fontweight='bold')
ax3.set_xticks(x_pos)
ax3.set_xticklabels(categories, fontsize=11)
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height/2,
             f'{int(height)}', ha='center', va='center', fontsize=12, fontweight='bold', color='white')
if blocked_count > 0:
    for i, bar in enumerate(bars2):
        height = bar.get_height()
        if height > 0:
            ax3.text(bar.get_x() + bar.get_width()/2., deployed_counts[i] + height/2,
                     f'{int(height)}', ha='center', va='center', fontsize=12, fontweight='bold', color='white')

# Plot 4: Deployment Status Over Drift
ax4 = axes[1, 1]
deployed_mask = np.array(deployed_with)
blocked_mask = ~deployed_mask

if np.any(deployed_mask):
    ax4.scatter(np.array(drift_vals)[deployed_mask], np.array(conf_with)[deployed_mask], 
                c='green', s=100, alpha=0.6, label='Deployed (Safe)', marker='o', edgecolors='darkgreen', linewidth=1.5)
if np.any(blocked_mask):
    ax4.scatter(np.array(drift_vals)[blocked_mask], np.array(conf_with)[blocked_mask], 
                c='red', s=100, alpha=0.6, label='Blocked (Unsafe)', marker='x', linewidth=2)

ax4.axhline(y=0.6, color='orange', linestyle='--', label='Confidence Threshold', linewidth=2)
ax4.set_xlabel('Drift Level', fontsize=12)
ax4.set_ylabel('Model Confidence', fontsize=12)
ax4.set_title('Safety Library Deployment Decisions', fontsize=13, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Print summary statistics
print("=" * 60)
print("COMPARISON SUMMARY: WITH vs WITHOUT AI Safety Library")
print("=" * 60)

print("\n📊 WITHOUT Safety Library:")
print(f"   ✓ Predictions Deployed: {deployed_count_without}/{num_samples} (100%)")
print(f"   ✗ Unsafe Predictions Deployed: Unknown (no safety checks)")
print(f"   ⚠️  Risk of deploying degraded predictions: HIGH")

print("\n🛡️  WITH Safety Library:")
print(f"   ✓ Safe Predictions Deployed: {deployed_count_with}/{num_samples} ({100*deployed_count_with/num_samples:.1f}%)")
print(f"   ✗ Unsafe Predictions Blocked: {blocked_count}/{num_samples} ({100*blocked_count/num_samples:.1f}%)")
print(f"   ⚠️  Risk of deploying degraded predictions: LOW")

print("\n📈 Key Insights:")
avg_risk_deployed = np.mean([r['risk_score'] for r in results_with_library if r['deployed']])
avg_risk_blocked = np.mean([r['risk_score'] for r in results_with_library if not r['deployed']]) if blocked_count > 0 else 0

print(f"   • Average risk of deployed predictions: {avg_risk_deployed:.4f}")
if blocked_count > 0:
    print(f"   • Average risk of blocked predictions: {avg_risk_blocked:.4f}")
    print(f"   • Safety improvement: {blocked_count} risky predictions prevented")
else:
    print(f"   • All predictions passed safety checks")

print("\n" + "=" * 60)